## 1) 데이터 로드하기
https://github.com/e9t/nsmc

ko_data.csv 파일은 구글 드라이버에 저장

In [1]:
#구글 드라이브 연동하기
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
%cd /gdrive/My Drive/Colab Notebooks/data/

/gdrive/My Drive/Colab Notebooks/data


In [3]:
%ls

 전국신규민간아파트분양가격동향/   서울_범죄_2018년.xlsx   friends_test.json
 01_CCTV_in_Seoul2.csv             body_cp949.txt          friends_train.json
'01. CCTV_in_Seoul.csv'            body.csv                ko_data.csv
 01_population_in_Seoul.xlsx       friends_dev.json


In [4]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 1.4MB 6.1MB/s 
     |████████████████████████████████| 2.9MB 35.1MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f00814a04371441bc2c4d8d8392d95a2f92d07e4b4fb87b4da84db1d556b1790
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [6]:
# 네이버 영화 리뷰 데이터 가져오기
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 8.32 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [8]:
# train/test set 데이터 로드

train = pd.read_csv("nsmc/ratings.txt", sep='\t')
test = pd.read_csv("/gdrive/My Drive/Colab Notebooks/data/ko_data.csv", encoding='CP949')

In [9]:
print('훈련용 리뷰 개수 :',len(train)) # 훈련용 리뷰 개수 출력

훈련용 리뷰 개수 : 200000


In [10]:
train[:5] # 상위 5개 출력 label : 긍정 (1), 부정 (0)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [11]:
print('테스트용 리뷰 개수 :',len(test)) # 테스트용 리뷰 개수 출력

테스트용 리뷰 개수 : 11187


In [12]:
test[:5] # 상위 5개 출력 

,Id,Sentence
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


# 2) 전처리 - 훈련셋

In [13]:
MAX_LEN = 128

def getInputs(dataset):
  data = dataset.copy(deep=True)

  if 'document' in data.columns:
    sentences = data['document']
  else:
    sentences = data['Sentence']

  sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
  
  tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator", do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

In [14]:
def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.index.tolist()
  return torch.tensor(input_index)

In [15]:
labels = train['label'].values
ratings_inputs, ratings_masks = getInputs(train)
test_inputs, test_masks = getInputs(test)

In [16]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(ratings_inputs, labels, random_state=2018, test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(ratings_masks, ratings_inputs, random_state=2018, test_size=0.1)

In [17]:
# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)			

test_index = getIndex(test)
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)

In [19]:
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_index, test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 3) 모델 생성

In [20]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [21]:
# ELECTRA 모델 생성

model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v2-discriminator", num_labels = 2)
model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [22]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [23]:

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [25]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로직과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  5,625.    Elapsed: 0:03:18.
  Batch 1,000  of  5,625.    Elapsed: 0:06:35.
  Batch 1,500  of  5,625.    Elapsed: 0:09:52.
  Batch 2,000  of  5,625.    Elapsed: 0:13:10.
  Batch 2,500  of  5,625.    Elapsed: 0:16:27.
  Batch 3,000  of  5,625.    Elapsed: 0:19:44.
  Batch 3,500  of  5,625.    Elapsed: 0:23:01.
  Batch 4,000  of  5,625.    Elapsed: 0:26:18.
  Batch 4,500  of  5,625.    Elapsed: 0:29:35.
  Batch 5,000  of  5,625.    Elapsed: 0:32:53.
  Batch 5,500  of  5,625.    Elapsed: 0:36:09.

  Average training loss: 0.29
  Training epcoh took: 0:36:59

Running Validation...
  Accuracy: 0.89
  Validation took: 0:01:17

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  5,625.    Elapsed: 0:03:17.
  Batch 1,000  of  5,625.    Elapsed: 0:06:34.
  Batch 1,500  of  5,625.    Elapsed: 0:09:51.
  Batch 2,000  of  5,625.    Elapsed: 0:13:07.
  Batch 2,500  of  5,625.    Elapsed: 0:16:24.
  Batch 3,000  of  5,625.    Elap

## 4) test set 평가

In [26]:
tmp_test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)
test_result = test.copy(deep = True)
test_result['Predicted'] = 'default'
classes = [0, 1]

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(tmp_test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = classes[np.argmax(logits)]
    

    nb_eval_steps += 1

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  Batch   100  of    350.    Elapsed: 0:00:01.
  Batch   200  of    350.    Elapsed: 0:00:02.
  Batch   300  of    350.    Elapsed: 0:00:04.
  Batch   400  of    350.    Elapsed: 0:00:05.
  Batch   500  of    350.    Elapsed: 0:00:06.
  Batch   600  of    350.    Elapsed: 0:00:07.
  Batch   700  of    350.    Elapsed: 0:00:08.
  Batch   800  of    350.    Elapsed: 0:00:09.
  Batch   900  of    350.    Elapsed: 0:00:11.
  Batch 1,000  of    350.    Elapsed: 0:00:12.
  Batch 1,100  of    350.    Elapsed: 0:00:13.
  Batch 1,200  of    350.    Elapsed: 0:00:14.
  Batch 1,300  of    350.    Elapsed: 0:00:15.
  Batch 1,400  of    350.    Elapsed: 0:00:16.
  Batch 1,500  of    350.    Elapsed: 0:00:18.
  Batch 1,600  of    350.    Elapsed: 0:00:19.
  Batch 1,700  of    350.    Elapsed: 0:00:20.
  Batch 1,800  of    350.    Elapsed: 0:00:21.
  Batch 1,900  of    350.    Elapsed: 0:00:22.
  Batch 2,000  of    350.    Elapsed: 0:00:23.
  Batch 2,100  of    350.    Elapsed: 0:00:25.
  Batch 2,200

In [27]:
test_result[:5]

,Id,Sentence,Predicted
0,0,정말 많이 울었던 영화입니다.,1
1,1,시간 낭비예요.,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,0
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,1
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,0


In [28]:
test_result.drop(['Sentence'], axis='columns', inplace=True)

In [29]:
test_result.set_index('Id',inplace=True)


In [30]:
test_result[:5]

,Predicted
Id,
0,1
1,0
2,0
3,1
4,0


In [31]:
test_csv = test_result.to_csv('test.csv')

In [32]:
from google.colab import files

files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>